# MNIST E2E on Kubeflow on Vanilla k8s

This example guides you through:
  
  1. Taking an example TensorFlow model and modifying it to support distributed training
  1. Serving the resulting model using TFServing
  1. Deploying and using a web-app that uses the model
  
## Requirements

  * You must be running Kubeflow 1.0 using the k8s istio config or the istio dex config.
 

## Prepare model

There is a delta between existing distributed mnist examples and what's needed to run well as a TFJob.

Basically, we must:

1. Add options in order to make the model configurable.
1. Use `tf.estimator.train_and_evaluate` to enable model exporting and serving.
1. Define serving signatures for model serving.

The resulting model is [model.py](model.py).

### Install Required Packages

Click `Kernel` -> `Restart` after your install new packages.

In [1]:
!pip install boto3 table_logger --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [ ]:
import notebook_setup
from importlib import reload
reload(notebook_setup)
notebook_setup.notebook_setup(platform='onprem')

pip installing requirements.txt


In [4]:
import k8s_util
# Force a reload of kubeflow; since kubeflow is a multi namespace module
# it looks like doing this in notebook_setup may not be sufficient
import kubeflow
reload(kubeflow)
from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubeflow.tfjob.api import tf_job_client as tf_job_client_module
from IPython.core.display import display, HTML
import yaml
from os import environ

## Configure external service credentials


## Step 1 - Pushing to DockerHub

Source documentation: [Kaniko docs](https://github.com/GoogleContainerTools/kaniko#pushing-to-docker-hub)

### Why do we need this?

Kaniko is used by fairing to build the model every time the notebook is run and deploy a fresh model.
The newly built image is pushed into the DOCKER_REGISTRY and pulled from there by subsequent resources.

### Configure docker credentials

Get your docker registry user and password encoded in base64 <br>

`echo -n USER:PASSWORD | base64` <br>

Create a config.json file with your Docker registry url and the previous generated base64 string <br>
```json
{
	"auths": {
		"https://index.docker.io/v1/": {
			"auth": "xxxxxxxxxxxxxxx"
		}
	}
}
```
```json
{
    "auths": {
        "https://small-sacha-644-harbor.app.small-sacha-644.bubble.superhub.io/library/": {
            "auth": "YWRtaW46QWRtaW4xMjM="
        }
    }
}
```

<br>

### Create a config-map in the namespace you're using with the docker config

`kubectl create --namespace ${NAMESPACE} configmap docker-config --from-file=<path to config.json>`
for example: <br>
"kubectl delete --namespace workspace configmap docker-config" <br>
"kubectl create --namespace workspace configmap docker-config --from-file=config.json" <br>

## Step 2 - Set DOCKER_REGISTRY

The **DOCKER_REGISTRY** variable is used to push the newly built image. <br>
Please change the variable to the registry for which you've configured credentials.

In [5]:
!kubectl delete configmap docker-config
!kubectl create configmap docker-config --from-file=/home/jovyan/.docker/config.json

configmap "docker-config" deleted
configmap/docker-config created


In [5]:
import logging
import os
import uuid
from importlib import reload
import boto3

In [6]:
from kubernetes import client as k8s_client
from kubernetes.client import rest as k8s_rest
from kubeflow import fairing   
from kubeflow.fairing import utils as fairing_utils
from kubeflow.fairing.builders import append
from kubeflow.fairing.deployers import job
from kubeflow.fairing.preprocessors import base as base_preprocessor

DOCKER_REGISTRY = environ['HARBOR_HOST'] + "/library"
#namespace = fairing_utils.get_current_k8s_namespace()
namespace = environ['THIS_NAMESPACE']

from kubernetes import client as k8s_client
from kubernetes.client.rest import ApiException

api_client = k8s_client.CoreV1Api()


s3_endpoint = environ['AWS_S3_ENDPOINT']
minio_service_endpoint = s3_endpoint
minio_endpoint = "http://"+s3_endpoint
minio_username = environ['AWS_ACCESS_KEY_ID']
minio_key = environ['AWS_SECRET_ACCESS_KEY']
minio_region = "us-east-1"

logging.info(f"Running in namespace {namespace}")
logging.info(f"Using docker registry {DOCKER_REGISTRY}")
logging.info(f"Using minio instance with endpoint '{s3_endpoint}'")

Running in namespace igor
Using docker registry stimulating-ladymaya-962-harbor.app.stimulating-ladymaya-962.bubble.superhub.io/library
Using minio instance with endpoint 'minio.kubeflow-data.svc.cluster.local:9000'


In [7]:
import logging
import os
import uuid
from importlib import reload
import boto3
from botocore.client import Config

s3 = boto3.client('s3', endpoint_url="http://"+environ['AWS_S3_ENDPOINT'], 
             aws_access_key_id=environ['AWS_ACCESS_KEY_ID'],
             aws_secret_access_key=environ['AWS_SECRET_ACCESS_KEY'],
             region_name='us-east-1')
for _ in s3.list_buckets()["Buckets"]:
  print(_["Name"])

bucket
kubeflow-us-east-1
stimulat-mnist


## Install Required Libraries

Import the libraries required to train this model.

In [8]:
import logging
import os
import uuid
from importlib import reload
import notebook_setup
reload(notebook_setup)
notebook_setup.notebook_setup(platform='onprem')

pip installing requirements.txt
Checkout kubeflow/tf-operator @9238906


In [9]:
import k8s_util
# Force a reload of kubeflow; since kubeflow is a multi namespace module
# it looks like doing this in notebook_setup may not be sufficient
import kubeflow
reload(kubeflow)
from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubeflow.tfjob.api import tf_job_client as tf_job_client_module
from IPython.core.display import display, HTML
import yaml

In [10]:
# TODO(https://github.com/kubeflow/fairing/issues/426): We should get rid of this once the default 
# Kaniko image is updated to a newer image than 0.7.0.
from kubeflow.fairing import constants
#constants.constants.KANIKO_IMAGE = "gcr.io/kaniko-project/executor:v0.14.0"
constants.constants.KANIKO_IMAGE = "gcr.io/kaniko-project/executor:v0.19.0"

In [11]:
from kubeflow.fairing.builders import cluster

# output_map is a map of extra files to add to the notebook.
# It is a map from source location to the location inside the context.
output_map =  {
    "Dockerfile.model": "Dockerfile",
    "model.py": "model.py"
}

preprocessor = base_preprocessor.BasePreProcessor(
    command=["python"], # The base class will set this.
    input_files=[],
    path_prefix="/app", # irrelevant since we aren't preprocessing any files
    output_map=output_map)

preprocessor.preprocess()

set()

In [12]:
# Use a Tensorflow image as the base image
# We use a custom Dockerfile 
from kubeflow.fairing.cloud.k8s import MinioUploader
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource

minio_uploader = MinioUploader(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)
minio_context_source = MinioContextSource(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)

In [13]:
print (minio_endpoint)
#docker push small-sacha-644-harbor.app.small-sacha-644.bubble.superhub.io/library/IMAGE[:TAG]

http://minio.kubeflow-data.svc.cluster.local:9000


In [14]:
cluster_builder = cluster.cluster.ClusterBuilder(registry=DOCKER_REGISTRY,
                                                 base_image="", # base_image is set in the Dockerfile
                                                 preprocessor=preprocessor,
                                                 image_name="mnist",
                                                 dockerfile_path="Dockerfile",
                                                 context_source=minio_context_source)
cluster_builder.build()
logging.info(f"Built image {cluster_builder.image_tag}")

Building image using cluster builder.
Creating docker context: /tmp/fairing_context_e7m3at6e
Dockerfile already exists in Fairing context, skipping...
Waiting for fairing-builder-zq86d-btskk to start...
Waiting for fairing-builder-zq86d-btskk to start...
Waiting for fairing-builder-zq86d-btskk to start...
Pod started running True


INFO[0000] Resolved base name tensorflow/tensorflow:1.15.2-py3 to tensorflow/tensorflow:1.15.2-py3
INFO[0000] Resolved base name tensorflow/tensorflow:1.15.2-py3 to tensorflow/tensorflow:1.15.2-py3
INFO[0000] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0001] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0002] Built cross stage deps: map[]
INFO[0002] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0002] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0003] Unpacking rootfs as cmd ADD model.py /opt/model.py requires it.
INFO[0023] Taking snapshot of full filesystem...
INFO[0032] Resolving paths
INFO[0036] Using files from context: [/kaniko/buildcontext/model.py]
INFO[0036] ADD model.py /opt/model.py
INFO[0036] RUN chmod +x /opt/model.py
INFO[0036] cmd: /bin/sh
INFO[0036] args: [-c chmod +x /opt/model.py]
INFO[0036] ENTRYPOINT ["/usr/local/bin/python"]
INFO[0036] CMD ["/opt/model.py"]
INFO[0036] Taking snapshot of full 

Built image stimulating-ladymaya-962-harbor.app.stimulating-ladymaya-962.bubble.superhub.io/library/mnist:6EF4A576


## Create a Minio Bucket

* Create a minio bucket to store our models and other results.

In [17]:
mnist_bucket = f"{DOCKER_REGISTRY}"[0:8]+"-mnist"
minio_uploader.create_bucket(mnist_bucket)
logging.info(f"Bucket {mnist_bucket} created or already exists")

Bucket stimulat-mnist created or already exists


## Distributed training

* We will train the model by using TFJob to run a distributed training job

### Training job parameters

In [18]:
train_name = f"mnist-train-{uuid.uuid4().hex[:4]}"
num_ps = 1
num_workers = 2
model_dir = f"s3://{mnist_bucket}/mnist"
export_path = f"s3://{mnist_bucket}/mnist/export" 
train_steps = 200
batch_size = 100
learning_rate = .01
image = cluster_builder.image_tag

In [19]:
train_spec = f"""apiVersion: kubeflow.org/v1
kind: TFJob
metadata:
  name: {train_name}  
spec:
  tfReplicaSpecs:
    Ps:
      replicas: {num_ps}
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          serviceAccount: default-editor
          containers:
          - name: tensorflow
            command:
            - python
            - /opt/model.py
            - --tf-model-dir={model_dir}
            - --tf-export-dir={export_path}
            - --tf-train-steps={train_steps}
            - --tf-batch-size={batch_size}
            - --tf-learning-rate={learning_rate}
            env:
            - name: S3_ENDPOINT
              value: {s3_endpoint}
            - name: AWS_ENDPOINT_URL
              value: {minio_endpoint}
            - name: AWS_REGION
              value: {minio_region}
            - name: BUCKET_NAME
              value: {mnist_bucket}
            - name: S3_USE_HTTPS
              value: "0"
            - name: S3_VERIFY_SSL
              value: "0"
            - name: AWS_ACCESS_KEY_ID
              value: {minio_username}
            - name: AWS_SECRET_ACCESS_KEY
              value: {minio_key}
            image: {image}
            workingDir: /opt
          restartPolicy: OnFailure
    Chief:
      replicas: 1
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          serviceAccount: default-editor
          containers:
          - name: tensorflow
            command:
            - python
            - /opt/model.py
            - --tf-model-dir={model_dir}
            - --tf-export-dir={export_path}
            - --tf-train-steps={train_steps}
            - --tf-batch-size={batch_size}
            - --tf-learning-rate={learning_rate}
            env:
            - name: S3_ENDPOINT
              value: {s3_endpoint}
            - name: AWS_ENDPOINT_URL
              value: {minio_endpoint}
            - name: AWS_REGION
              value: {minio_region}
            - name: BUCKET_NAME
              value: {mnist_bucket}
            - name: S3_USE_HTTPS
              value: "0"
            - name: S3_VERIFY_SSL
              value: "0"
            - name: AWS_ACCESS_KEY_ID
              value: {minio_username}
            - name: AWS_SECRET_ACCESS_KEY
              value: {minio_key}
            image: {image}
            workingDir: /opt
          restartPolicy: OnFailure
    Worker:
      replicas: 1
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          serviceAccount: default-editor
          containers:
          - name: tensorflow
            command:
            - python
            - /opt/model.py
            - --tf-model-dir={model_dir}
            - --tf-export-dir={export_path}
            - --tf-train-steps={train_steps}
            - --tf-batch-size={batch_size}
            - --tf-learning-rate={learning_rate}
            env:
            - name: S3_ENDPOINT
              value: {s3_endpoint}
            - name: AWS_ENDPOINT_URL
              value: {minio_endpoint}
            - name: AWS_REGION
              value: {minio_region}
            - name: BUCKET_NAME
              value: {mnist_bucket}
            - name: S3_USE_HTTPS
              value: "0"
            - name: S3_VERIFY_SSL
              value: "0"
            - name: AWS_ACCESS_KEY_ID
              value: {minio_username}
            - name: AWS_SECRET_ACCESS_KEY
              value: {minio_key}
            image: {image}
            workingDir: /opt
          restartPolicy: OnFailure
""" 

### Create the training job

* You could write the spec to a YAML file and then do `kubectl apply -f {FILE}`
* Since you are running in jupyter you will use the TFJob client
* You will run the TFJob in a namespace created by a Kubeflow profile
  * The namespace will be the same namespace you are running the notebook in
  * Creating a profile ensures the namespace is provisioned with service accounts and other resources needed for Kubeflow

In [20]:
tf_job_client = tf_job_client_module.TFJobClient()

In [21]:
tf_job_body = yaml.safe_load(train_spec)
tf_job = tf_job_client.create(tf_job_body, namespace=namespace)  

logging.info(f"Created job {namespace}.{train_name}")

Created job igor.mnist-train-688f


In [23]:
from kubeflow.tfjob import TFJobClient
tfjob_client = TFJobClient()
tfjob_client.wait_for_job(train_name, namespace=namespace, watch=True)

NAME                           STATE                TIME                          
mnist-train-688f               Created              2021-02-15T17:47:58Z          
mnist-train-688f               Created              2021-02-15T17:47:58Z          
mnist-train-688f               Running              2021-02-15T17:48:51Z          
mnist-train-688f               Running              2021-02-15T17:48:51Z          
mnist-train-688f               Succeeded            2021-02-15T17:48:58Z          


## Get TF Job logs

In [24]:
tfjob_client.get_logs(train_name, namespace=namespace)

The logs of Pod mnist-train-688f-chief-0:


W0215 17:48:52.994845 140422158473024 module_wrapper.py:139] From /opt/model.py:153: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0215 17:48:52.995030 140422158473024 module_wrapper.py:139] From /opt/model.py:153: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0215 17:48:52.996400 140422158473024 module_wrapper.py:139] From /opt/model.py:158: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:TF_CONFIG {"cluster":{"chief":["mnist-train-688f-chief-0.igor.svc:2222"],"ps":["mnist-train-688f-ps-0.igor.svc:2222"],"worker":["mnist-train-688f-worker-0.igor.svc:2222"]},"task":{"type":"chief","index":0},"environment":"cloud"}
I0215 17:48:52.996509 140422158473024 model.py:158] TF_CONFIG {"cluster":{"chief":["mnist-train-688f-chief-0.igor.svc:2222"],"ps":["mnist-train-688f-ps-0.igor.svc:2222"],"work

## Check the model in Minio

In [25]:
#TODO(swiftdiaries): Check object key for model specifically
from botocore.exceptions import ClientError

try:
    model_response = minio_uploader.client.list_objects(Bucket=mnist_bucket)
    # Minimal check to see if at least the bucket is created
    if model_response["ResponseMetadata"]["HTTPStatusCode"] == 200:
        logging.info(f"{model_dir} found in {mnist_bucket} bucket")
except ClientError as err:
    logging.error(err)

s3://stimulat-mnist/mnist found in stimulat-mnist bucket


## Deploy Tensorboard

In [53]:
tb_name = "mnist-tensorboard"
tb_deploy = f"""apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app: mnist-tensorboard
  name: {tb_name}
  namespace: {namespace}
spec:
  selector:
    matchLabels:
      app: mnist-tensorboard
  template:
    metadata:
      labels:
        app: mnist-tensorboard
        version: v1
    spec:
      serviceAccount: default-editor
      containers:
      - command:
        - /usr/local/bin/tensorboard
        - --logdir={model_dir}
        - --port=80
        image: tensorflow/tensorflow:1.15.2-py3
        env:
        - name: S3_ENDPOINT
          value: {s3_endpoint}
        - name: AWS_ENDPOINT_URL
          value: {minio_endpoint}
        - name: AWS_REGION
          value: {minio_region}
        - name: BUCKET_NAME
          value: {mnist_bucket}
        - name: S3_USE_HTTPS
          value: "0"
        - name: S3_VERIFY_SSL
          value: "0"
        - name: AWS_ACCESS_KEY_ID
          value: {minio_username}
        - name: AWS_SECRET_ACCESS_KEY
          value: {minio_key}  
        name: tensorboard
        ports:
        - containerPort: 80
"""
tb_service = f"""apiVersion: v1
kind: Service
metadata:
  labels:
    app: mnist-tensorboard
  name: {tb_name}
  namespace: {namespace}
spec:
  ports:
  - name: http-tb
    port: 80
    targetPort: 80
  selector:
    app: mnist-tensorboard
  type: ClusterIP
"""

tb_virtual_service = f"""apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: {tb_name}
  namespace: {namespace}
spec:
  gateways:
  - kubeflow/kubeflow-gateway
  hosts:
  - '*'
  http:
  - match:
    - uri:
        prefix: /mnist/{namespace}/tensorboard/
    rewrite:
      uri: /
    route:
    - destination:
        host: {tb_name}.{namespace}.svc.cluster.local
        port:
          number: 80
    timeout: 300s
"""

tb_specs = [tb_deploy, tb_service, tb_virtual_service]

In [54]:
k8s_util.apply_k8s_specs(tb_specs, k8s_util.K8S_CREATE_OR_REPLACE)

/home/jovyan/kubeflow-examples/mnist/k8s_util.py:55: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  spec = yaml.load(spec)
Created Deployment igor.mnist-tensorboard
Created Service igor.mnist-tensorboard
Created VirtualService mnist-tensorboard.mnist-tensorboard


[{'api_version': 'apps/v1',
  'kind': 'Deployment',
  'metadata': {'annotations': None,
               'cluster_name': None,
               'creation_timestamp': datetime.datetime(2021, 2, 15, 6, 22, 6, tzinfo=tzlocal()),
               'deletion_grace_period_seconds': None,
               'deletion_timestamp': None,
               'finalizers': None,
               'generate_name': None,
               'generation': 1,
               'initializers': None,
               'labels': {'app': 'mnist-tensorboard'},
               'managed_fields': None,
               'name': 'mnist-tensorboard',
               'namespace': 'igor',
               'owner_references': None,
               'resource_version': '627423',
               'self_link': '/apis/apps/v1/namespaces/igor/deployments/mnist-tensorboard',
               'uid': '489d6142-bc5b-4ba9-9dec-2f5f411923fc'},
  'spec': {'min_ready_seconds': None,
           'paused': None,
           'progress_deadline_seconds': 600,
           'rep

## Get Tensorboard URL

Run this with the appropriate RBAC permissions <br>

In [56]:
kubeflow_url = environ["KUBEFLOW_HOST"]
endpoint = kubeflow_url 
if endpoint:    
    vs = yaml.safe_load(tb_virtual_service)
    path= vs["spec"]["http"][0]["match"][0]["uri"]["prefix"]
    tb_endpoint = endpoint + path
    display(HTML(f"TensorBoard UI is at http://{tb_endpoint}"))

## Serve the model

* Deploy the model using tensorflow serving
* We need to create
  1. A Kubernetes Deployment
  1. A Kubernetes service
  1. (Optional) Create a configmap containing the prometheus monitoring config

In [26]:
namespace

'igor'

In [27]:
export_path

's3://stimulat-mnist/mnist/export'

In [28]:
deploy_name = "mnist-model"
model_base_path = export_path

# The web ui defaults to mnist-service so if you change it you will
# need to change it in the UI as well to send predictions to the mode
model_service = "mnist-service"

deploy_spec = f"""apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app: mnist
  name: {deploy_name}
  namespace: {namespace}
spec:
  selector:
    matchLabels:
      app: mnist-model
  template:
    metadata:
      # TODO(jlewi): Right now we disable the istio side car because otherwise ISTIO rbac will prevent the
      # UI from sending RPCs to the server. We should create an appropriate ISTIO rbac authorization
      # policy to allow traffic from the UI to the model servier.
      # https://istio.io/docs/concepts/security/#target-selectors
      annotations:        
        sidecar.istio.io/inject: "false"
      labels:
        app: mnist-model
        version: v1
    spec:
      serviceAccount: default-editor
      containers:
      - args:
        - --port=9000
        - --rest_api_port=8500
        - --model_name=mnist
        - --model_base_path={model_base_path}
        command:
        - /usr/bin/tensorflow_model_server
        env:
        - name: modelBasePath
          value: {model_base_path}
        - name: S3_ENDPOINT
          value: {s3_endpoint}
        - name: AWS_ENDPOINT_URL
          value: {minio_endpoint}
        - name: AWS_REGION
          value: {minio_region}
        - name: BUCKET_NAME
          value: {mnist_bucket}
        - name: S3_USE_HTTPS
          value: "0"
        - name: S3_VERIFY_SSL
          value: "0"
        - name: AWS_ACCESS_KEY_ID
          value: {minio_username}
        - name: AWS_SECRET_ACCESS_KEY
          value: {minio_key}  
        image: tensorflow/serving:1.15.0
        imagePullPolicy: IfNotPresent
        livenessProbe:
          initialDelaySeconds: 30
          periodSeconds: 30
          tcpSocket:
            port: 9000
        name: mnist
        ports:
        - containerPort: 9000
        - containerPort: 8500
        resources:
          limits:
            cpu: "4"
            memory: 4Gi
          requests:
            cpu: "1"
            memory: 1Gi
        volumeMounts:
        - mountPath: /var/config/
          name: model-config
      volumes:
      - configMap:
          name: {deploy_name}
        name: model-config
"""

service_spec = f"""apiVersion: v1
kind: Service
metadata:
  annotations:    
    prometheus.io/path: /monitoring/prometheus/metrics
    prometheus.io/port: "8500"
    prometheus.io/scrape: "true"
  labels:
    app: mnist-model
  name: {model_service}
  namespace: {namespace}
spec:
  ports:
  - name: grpc-tf-serving
    port: 9000
    targetPort: 9000
  - name: http-tf-serving
    port: 8500
    targetPort: 8500
  selector:
    app: mnist-model
  type: ClusterIP
"""

monitoring_config = f"""kind: ConfigMap
apiVersion: v1
metadata:
  name: {deploy_name}
  namespace: {namespace}
data:
  monitoring_config.txt: |-
    prometheus_config: {{
      enable: true,
      path: "/monitoring/prometheus/metrics"
    }}
"""

model_specs = [deploy_spec, service_spec, monitoring_config]

In [29]:
k8s_util.apply_k8s_specs(model_specs, k8s_util.K8S_CREATE_OR_REPLACE)     

/home/jovyan/kubeflow-examples/mnist/k8s_util.py:55: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  spec = yaml.load(spec)
Deleted Deployment igor.mnist-model
Created Deployment igor.mnist-model
Deleted Service igor.mnist-service
Created Service igor.mnist-service
Deleted ConfigMap igor.mnist-model
Created ConfigMap igor.mnist-model


[{'api_version': 'apps/v1',
  'kind': 'Deployment',
  'metadata': {'annotations': None,
               'cluster_name': None,
               'creation_timestamp': datetime.datetime(2021, 2, 15, 17, 49, 52, tzinfo=tzlocal()),
               'deletion_grace_period_seconds': None,
               'deletion_timestamp': None,
               'finalizers': None,
               'generate_name': None,
               'generation': 1,
               'initializers': None,
               'labels': {'app': 'mnist'},
               'managed_fields': None,
               'name': 'mnist-model',
               'namespace': 'igor',
               'owner_references': None,
               'resource_version': '825133',
               'self_link': '/apis/apps/v1/namespaces/igor/deployments/mnist-model',
               'uid': '285dd9b7-f271-41f3-bf1f-808d981f053b'},
  'spec': {'min_ready_seconds': None,
           'paused': None,
           'progress_deadline_seconds': 600,
           'replicas': 1,
           

## Deploy the mnist UI

* We will now deploy the UI to visual the mnist results
* Note: This is using a prebuilt and public docker image for the UI
* This example is simplified for training purposes. For production deployments follow CI/CD best practices.

In [30]:
ui_name = "mnist-ui"
ui_deploy = f"""apiVersion: apps/v1
kind: Deployment
metadata:
  name: {ui_name}
  namespace: {namespace}
spec:
  replicas: 1
  selector:
    matchLabels:
      app: mnist-web-ui
  template:
    metadata:
      labels:
        app: mnist-web-ui
    spec:
      containers:
      - image: gcr.io/kubeflow-examples/mnist/web-ui:v20190112-v0.2-142-g3b38225
        name: web-ui
        ports:
        - containerPort: 5000        
      serviceAccount: default-editor
"""

ui_service = f"""apiVersion: v1
kind: Service
metadata:
  annotations:
  name: {ui_name}
  namespace: {namespace}
spec:
  ports:
  - name: http-mnist-ui
    port: 80
    targetPort: 5000
  selector:
    app: mnist-web-ui
  type: ClusterIP
"""

ui_virtual_service = f"""apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: {ui_name}
  namespace: {namespace}
spec:
  gateways:
  - kubeflow/kubeflow-gateway
  hosts:
  - '*'
  http:
  - match:
    - uri:
        prefix: /mnist/{namespace}/ui/
    rewrite:
      uri: /
    route:
    - destination:
        host: {ui_name}.{namespace}.svc.cluster.local
        port:
          number: 80
    timeout: 300s
"""

ui_specs = [ui_deploy, ui_service, ui_virtual_service]

In [31]:
k8s_util.apply_k8s_specs(ui_specs, k8s_util.K8S_CREATE_OR_REPLACE)     

Deleted Deployment igor.mnist-ui
Created Deployment igor.mnist-ui
Deleted Service igor.mnist-ui
Created Service igor.mnist-ui
Deleted VirtualService igor.mnist-ui
Created VirtualService mnist-ui.mnist-ui


[{'api_version': 'apps/v1',
  'kind': 'Deployment',
  'metadata': {'annotations': None,
               'cluster_name': None,
               'creation_timestamp': datetime.datetime(2021, 2, 15, 17, 50, 5, tzinfo=tzlocal()),
               'deletion_grace_period_seconds': None,
               'deletion_timestamp': None,
               'finalizers': None,
               'generate_name': None,
               'generation': 1,
               'initializers': None,
               'labels': None,
               'managed_fields': None,
               'name': 'mnist-ui',
               'namespace': 'igor',
               'owner_references': None,
               'resource_version': '825236',
               'self_link': '/apis/apps/v1/namespaces/igor/deployments/mnist-ui',
               'uid': 'e250e604-008c-48f5-8afc-e946b7b63b21'},
  'spec': {'min_ready_seconds': None,
           'paused': None,
           'progress_deadline_seconds': 600,
           'replicas': 1,
           'revision_history_l

## Access the  web UI


In [32]:
app_url = environ["KUBEFLOW_HOST"]+ "/mnist/" + namespace + "/ui/"
logging.info(f"Web UI URL: http://{app_url}")

Web UI URL: http://kubeflow.needy-falcon-924.bubble.superhub.io/mnist/igor/ui/


## Serve the model with Seldon

* Deploy the model using Seldon
* We need to create
  1. A Kubernetes Deployment
  2. A Kubernetes service
  3. (Optional) Create a configmap containing the prometheus monitoring config

In [90]:
! pip install --user --upgrade seldon-core protobuf

     |████████████████████████████████| 133kB 23.3MB/s eta 0:00:01
Requirement already up-to-date: protobuf in /home/jovyan/.local/lib/python3.6/site-packages (3.14.0)
     |████████████████████████████████| 51kB 11.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 16.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.2MB/s eta 0:00:01
     |████████████████████████████████| 92kB 18.0MB/s eta 0:00:01
     |████████████████████████████████| 102kB 18.9MB/s ta 0:00:01
     |████████████████████████████████| 61kB 13.8MB/s eta 0:00:01
     |████████████████████████████████| 92kB 18.8MB/s eta 0:00:01
  Created wheel for opentracing: filename=opentracing-2.4.0-cp36-none-any.whl size=51877 sha256=e0d5d4198d7e880041e25ed866a740dbaf513b71fc642763bda13ef80a7d3f8c
  Stored in directory: /home/jovyan/.cache/pip/wheels/27/b6/a0/d0309988a0dd5623c34469b151e4d7b0e6271b28a8bcccb440
  Created wheel for Flask-OpenTracing: filename=Flask_OpenTracing-1.1.0-cp36-none-any.whl size=

  Created wheel for threadloop: filename=threadloop-1.0.2-cp36-none-any.whl size=4265 sha256=21db4ff078690e7d1164e395b8b162b19649018330219ba998bfb20f8fa6c56c
  Stored in directory: /home/jovyan/.cache/pip/wheels/d7/7a/30/d212623a4cd34f6cce400f8122b1b7af740d3440c68023d51f
  Created wheel for thrift: filename=thrift-0.13.0-cp36-cp36m-linux_x86_64.whl size=346213 sha256=27b4013a109112b0779426616e8be0339b940ca7ef068f8e8b7eda62b3efcb79
  Stored in directory: /home/jovyan/.cache/pip/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
Successfully built opentracing Flask-OpenTracing jaeger-client grpcio-reflection threadloop thrift
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
s3 = boto3.client('s3', endpoint_url="http://"+environ['AWS_S3_ENDPOINT'], 
             aws_access_key_id=environ['AWS_ACCESS_KEY_ID'],
             aws_secret_access_key=environ['AWS_SECRET_ACCESS_KEY'],
             region_name='us-east-1')

NameError: name 'boto3' is not defined

In [72]:
s3.list_buckets()

{'ResponseMetadata': {'RequestId': '1663D88D98BB3ACB',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '557',
   'content-security-policy': 'block-all-mixed-content',
   'content-type': 'application/xml',
   'server': 'envoy',
   'vary': 'Origin',
   'x-amz-request-id': '1663D88D98BB3ACB',
   'x-xss-protection': '1; mode=block',
   'date': 'Mon, 15 Feb 2021 06:39:53 GMT',
   'x-envoy-upstream-service-time': '1'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'bucket',
   'CreationDate': datetime.datetime(2021, 2, 13, 20, 21, 32, 403000, tzinfo=tzlocal())},
  {'Name': 'kubeflow-us-east-1',
   'CreationDate': datetime.datetime(2021, 2, 15, 6, 18, 42, 46000, tzinfo=tzlocal())},
  {'Name': 'stimulat-mnist',
   'CreationDate': datetime.datetime(2021, 2, 15, 6, 20, 5, 692000, tzinfo=tzlocal())}],
 'Owner': {'DisplayName': '',
  'ID': '02d6176db174dc93cb1b899f7c6078f08654445fe8cf1b6ce98d8855f66bdbf4'}}

In [74]:
secret_spec = f"""apiVersion: v1
kind: Secret
metadata:
metadata:
  name: seldon-init-container-secret
  namespace: {namespace}
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: {minio_username}
  AWS_SECRET_ACCESS_KEY: {minio_key}
  AWS_ENDPOINT_URL: {minio_endpoint}
  USE_SSL: "false"
"""

minio_specs = [secret_spec]      

In [75]:
k8s_util.apply_k8s_specs(minio_specs, k8s_util.K8S_CREATE_OR_REPLACE)

Created Secret igor.seldon-init-container-secret


[{'api_version': 'v1',
  'data': {'AWS_ACCESS_KEY_ID': 'ZTI4MjgzYjhjNTNkNDEyYzliMDMwOTJlNThkYTUxZTA=',
           'AWS_ENDPOINT_URL': 'aHR0cDovL21pbmlvLmt1YmVmbG93LWRhdGEuc3ZjLmNsdXN0ZXIubG9jYWw6OTAwMA==',
           'AWS_SECRET_ACCESS_KEY': 'ZDIxYWRlMTE3YjhlNDQ1MDhmMTBlNWFlNzk2Y2Y5MDI=',
           'USE_SSL': 'ZmFsc2U='},
  'kind': 'Secret',
  'metadata': {'annotations': None,
               'cluster_name': None,
               'creation_timestamp': datetime.datetime(2021, 2, 15, 6, 44, 1, tzinfo=tzlocal()),
               'deletion_grace_period_seconds': None,
               'deletion_timestamp': None,
               'finalizers': None,
               'generate_name': None,
               'generation': None,
               'initializers': None,
               'labels': None,
               'managed_fields': None,
               'name': 'seldon-init-container-secret',
               'namespace': 'igor',
               'owner_references': None,
               'resource_version': '63376

In [76]:
export_path

's3://stimulat-mnist/mnist/export'

In [77]:
#export_path="s3://mnist/mnist"

In [78]:
deploy_name = "tfserving"
model_base_path = export_path

# The web ui defaults to mnist2-service so if you change it you will
# need to change it in the UI as well to send predictions to the mode
model_service = "mnist2-service"

deploy_spec = f"""apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: {deploy_name}
  namespace: {namespace}
  annotations:        
    sidecar.istio.io/inject: "false"
spec:
  name: mnist
  serviceAccount: default-editor
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: {model_base_path}
      envSecretRefName: seldon-init-container-secret
      name: mnist
      parameters:
        - name: signature_name
          type: STRING
          value: serving_default
        - name: model_name
          type: STRING
          value: mnist
        - name: model_input
          type: STRING
          value: images
        - name: model_output
          type: STRING
          value: scores     
    name: default
    replicas: 1
"""

service_spec = f"""apiVersion: v1
kind: Service
metadata:
  annotations:    
    prometheus.io/path: /monitoring/prometheus/metrics
    prometheus.io/port: "8500"
    prometheus.io/scrape: "true"
  labels:
    app: mnist-model
  name: {model_service}
  namespace: {namespace}
spec:
  ports:
  - name: grpc-tf-serving
    port: 9000
    targetPort: 9000
  - name: http-tf-serving
    port: 8500
    targetPort: 8500
  selector:
    app: mnist-model
  type: ClusterIP
"""

monitoring_config = f"""kind: ConfigMap
apiVersion: v1
metadata:
  name: {deploy_name}
  namespace: {namespace}
data:
  monitoring_config.txt: |-
    prometheus_config: {{
      enable: true,
      path: "/monitoring/prometheus/metrics"
    }}
"""

model_specs = [deploy_spec, service_spec, monitoring_config]    

In [79]:
k8s_util.apply_k8s_specs(model_specs, k8s_util.K8S_CREATE_OR_REPLACE) 

Created SeldonDeployment tfserving.tfserving
Created Service igor.mnist2-service
Created ConfigMap igor.tfserving


[{'apiVersion': 'machinelearning.seldon.io/v1alpha2',
  'kind': 'SeldonDeployment',
  'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
   'creationTimestamp': '2021-02-15T06:45:28Z',
   'generation': 1,
   'name': 'tfserving',
   'namespace': 'igor',
   'resourceVersion': '634184',
   'selfLink': '/apis/machinelearning.seldon.io/v1alpha2/namespaces/igor/seldondeployments/tfserving',
   'uid': '87df3a4b-7552-4d40-b519-4fb9f282d6b5'},
  'spec': {'name': 'mnist',
   'predictors': [{'componentSpecs': [{'metadata': {'creationTimestamp': '2021-02-15T06:45:28Z'},
       'spec': {'containers': [{'image': 'seldonio/tfserving-proxy:1.5.0',
          'name': 'mnist',
          'ports': [{'containerPort': 6000,
            'name': 'metrics',
            'protocol': 'TCP'}],
          'resources': {},
          'volumeMounts': [{'mountPath': '/etc/podinfo',
            'name': 'seldon-podinfo'}]}]}}],
     'engineResources': {},
     'graph': {'endpoint': {'grpcPort': 9500,
       

Add admin permissions to the notebook user using the following command: <br>`kubectl create clusterrolebinding workspace-cluster-admin --clusterrole=cluster-admin --serviceaccount=workspace:default-editor`

In [80]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=tfserving -o jsonpath='{.items[0].metadata.name}')
#deployment "tfserving-default-0-mnist-model" successfully rolled out

Waiting for deployment "tfserving-default-0-mnist" rollout to finish: 0 of 1 updated replicas are available...
deployment "tfserving-default-0-mnist" successfully rolled out


Get a token from the Dex gateway. At present as Dex does not support curl password credentials you will need to get it from your browser logged into the cluster. Open up a browser console and run `document.cookie`

In [91]:
TOKEN="MTYxMzM2OTI4MHxOd3dBTkU0MVEwaFhOVFZFUjB4SldGZEpTelJZUmt4T05raE9Va3MxTlVKQk1rdFZVRm8yUjFoTE5razNURGRaV2tkQlR6SlVTMEU9fCEp74dNjfQWPer__wMfU3lObjL58aUoShIC4yQRpRfF"

In [98]:
from seldon_core.seldon_client import SeldonClient, SeldonChannelCredentials, SeldonCallCredentials
host = kubeflow_url
ISTIO_GATEWAY=host
deployment_name = "tfserving"
transport="rest"

sc = SeldonClient(deployment_name=deployment_name,namespace=namespace,gateway_endpoint=ISTIO_GATEWAY,debug=False,
                 channel_credentials=SeldonChannelCredentials(verify=False),
                 call_credentials=SeldonCallCredentials(token=TOKEN))

In [99]:
r = sc.predict(gateway="istio",transport="rest",shape=(1,784))
print(r)
assert(r.success==True)


Success:False message:404:Not Found
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.744299398502212
    values: 0.2717148348158829
    values: 0.6376587571884251
    values: 0.2360589963558326
    values: 0.762329456850618
    values: 0.7853362242607471
    values: 0.8689150217699292
    values: 0.38866795663798037
    values: 0.15046378636790192
    values: 0.7824764102959192
    values: 0.3418876535739598
    values: 0.7430251373203408
    values: 0.41668988344872715
    values: 0.4133190575411557
    values: 0.9543291672835856
    values: 0.8289548563906406
    values: 0.6985469105348215
    values: 0.9807539221578262
    values: 0.5230015718538539
    values: 0.435283193073689
    values: 0.9429493620631536
    values: 0.6574265028361329
    values: 0.6471220552886202
    values: 0.4355182138661393
    values: 0.9455793152670438
    values: 0.3251655298513093
    values: 0.5165873011375982
    values: 0.9727207993749689
    values: 0.43213901343952954


/home/jovyan/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


AssertionError: 

In [100]:
print (f"Inference request successful: {r.success}")

Inference request successful: False
